# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [18]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [19]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,san antonio de pale,-1.4014,5.6325,79.74,70,90,10.54,GQ,1684706241
1,1,utrik,11.2278,169.8474,82.02,82,68,15.10,MH,1684706241
2,2,chinacota,7.6073,-72.6011,74.73,98,100,0.36,CO,1684706241
3,3,sudak,44.8500,34.9667,57.69,95,100,11.52,UA,1684706038
4,4,igarka,67.4667,86.5833,34.18,98,100,8.90,RU,1684706241


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [20]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    c='Humidity',
    size='Humidity',
    hover_cols=['City'],
    geo=True,
    tiles='CartoLight',
    frame_height=500
)

# Display the map
map_plot

:Overlay
   .Tiles.I   :Tiles   [x,y]
   .Scatter.I :Scatter   [Lng]   (Lat,Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [21]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
Narrow_down_cities = (
    (city_data_df['Max Temp'] > 70) &  # Temperature greater than 70 degrees Fahrenheit
    (city_data_df['Max Temp'] < 80) &  # Temperature less than 80 degrees Fahrenheit
    (city_data_df['Wind Speed'] < 10) &  # Wind speed less than 10 mph
    (city_data_df['Cloudiness'] == 0)  # No clouds
)

# Drop any rows with null values
# YOUR CODE HERE
ideal_cities_df = city_data_df[Narrow_down_cities].dropna()

# Display sample data
# YOUR CODE HERE
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
135,135,saint-pierre,-21.3393,55.4781,73.08,68,0,6.91,RE,1684706269
206,206,danville,36.5860,-79.3950,78.35,46,0,5.75,US,1684706282
224,224,tarija,-21.5355,-64.7296,78.71,19,0,9.22,BO,1684706286
244,244,jefferson city,38.5767,-92.1735,78.71,34,0,5.75,US,1684706141
267,267,miles city,46.4083,-105.8406,76.17,35,0,4.61,US,1684706296


### Step 3: Create a new DataFrame called `hotel_df`.

In [22]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
135,saint-pierre,RE,-21.3393,55.4781,68,
206,danville,US,36.5860,-79.3950,46,
224,tarija,BO,-21.5355,-64.7296,19,
244,jefferson city,US,38.5767,-92.1735,34,
267,miles city,US,46.4083,-105.8406,35,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [25]:
# Set parameters to search for a hotel
radius = 10000  # 10,000 meters
params = {
    "type": "hotel",
    "limit": 1,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = {
        "around": f"{latitude},{longitude}",
        "radius": radius
    }
    params["bias"] = f"proximity:{latitude},{longitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saint-pierre - nearest hotel: No hotel found
danville - nearest hotel: No hotel found
tarija - nearest hotel: No hotel found
jefferson city - nearest hotel: No hotel found
miles city - nearest hotel: No hotel found
maintirano - nearest hotel: No hotel found
cam ranh - nearest hotel: No hotel found
al jawf - nearest hotel: No hotel found
shamokin - nearest hotel: No hotel found
pamandzi - nearest hotel: No hotel found
quesnel - nearest hotel: No hotel found
demopolis - nearest hotel: No hotel found
el gouna - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
135,saint-pierre,RE,-21.3393,55.4781,68,No hotel found
206,danville,US,36.5860,-79.3950,46,No hotel found
224,tarija,BO,-21.5355,-64.7296,19,No hotel found
244,jefferson city,US,38.5767,-92.1735,34,No hotel found
267,miles city,US,46.4083,-105.8406,35,No hotel found
381,maintirano,MG,-18.0667,44.0167,83,No hotel found
427,cam ranh,VN,11.9214,109.1591,83,No hotel found
435,al jawf,SA,29.5000,38.7500,23,No hotel found
443,shamokin,US,40.7890,-76.5589,36,No hotel found
482,pamandzi,YT,-12.7967,45.2794,89,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [26]:
%%capture --no-display

# Configure the map plot
hover_info = ['City', 'Hotel Name', 'Country']  # Specify the columns for hover information
hotel_map = hotel_df.hvplot.points(x='Lng', y='Lat', c='Humidity', hover_cols=hover_info, 
                                   hover=True, alpha=0.8, cmap='coolwarm', size='Humidity',
                                   title='Hotel Map', xlabel='Longitude', ylabel='Latitude')

# Display the map
hotel_map

:Points   [Lng,Lat]   (Humidity,City,Hotel Name,Country)